In [1]:
import mea_analysis_pipeline as msp
import helper_functions as helper

In [ ]:
import importlib
importlib.reload(msp)
importlib.reload(helper)

In [ ]:
file_path= '/home/mmpatil/Documents/spikesorting/Data/activity_16757_1min_fullsccan/data.raw.h5' #network data from chip 16848
   
number_of_configurations = 3# need to know it while recording

time_in_s = 59

result = msp.routine_sequential(file_path,number_of_configurations,time_in_s)

In [2]:
file_path= '/mnt/harddrive-2/mmpatil/Data/data.raw.h5' #network data from chip 16848
   
rec_number= 10# need [0],locations[8][1]to know it while recording

time_in_s = 299

msp.process_block(rec_number,file_path,time_in_s)

Processing recording: rec0010
Sampling frequency: 20000.0
Number of channels: 1008
Number of segments: 1
total_recording: 300.06 s
Starting container
Installing spikeinterface==0.97.0 in kilosort3-maxwellcomplib:latest
Installing extra requirements: ['neo', 'h5py']
Running kilosort3 sorter inside kilosort3-maxwellcomplib:latest
Stopping container


extract waveforms memmap:   0%|          | 0/299 [00:00<?, ?it/s]

Sort and extract waveforms takes 639.5193604230008


/home/mmpatil/.local/lib/python3.10/site-packages/spikeinterface/qualitymetrics/misc_metrics.py:514: UserWarning: Units [2, 4, 6, 7, 18, 22, 23, 24, 26, 32, 33, 46, 49, 50, 53, 54, 60, 64, 65, 67, 70, 71, 72, 74, 76, 78, 84, 86, 92, 98, 99, 103, 104, 105, 107, 110, 115, 118, 119, 120, 121, 123, 125, 126, 127, 131, 132, 143, 144, 146, 150, 156, 158, 159, 160, 167, 171, 172, 175, 176, 177, 178, 179, 181, 182, 183, 191, 199, 203, 209, 211, 212, 213, 215, 217, 218, 222, 226] have too few spikes and amplitude_cutoff is set to NaN
  warnings.warn(f"Units {nan_units} have too few spikes and "
/home/mmpatil/.local/lib/python3.10/site-packages/spikeinterface/postprocessing/template_tools.py:19: DeprecationWarning: The spikeinterface.postprocessing.template_tools is submodule is deprecated.Use spikeinterface.core.template_tools instead
  _warn()


extract waveforms memmap:   0%|          | 0/299 [00:00<?, ?it/s]

In [ ]:
sorting1min = msp.get_kilosort_result('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/sorter_workspace/block_rec0007/kilosort3_rec0007/sorter_output')
waveforms1min = msp.get_waveforms_result(sorting1min,'/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/sorter_workspace/block_rec0007/waveforms_rec0007')
waveforms_good = msp.get_waveforms_result(sorting1min,'/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/sorter_workspace/block_rec0007/waveforms_good_rec0007')

In [ ]:
non_violated_units = msp.remove_violated_units(sorting1min,waveforms1min)
print(non_violated_units)
template_channel_dict = msp.get_unique_templates_channels(non_violated_units,waveforms1min)
non_redundant_templates = list(template_channel_dict.keys())
print(non_redundant_templates)

In [ ]:

import os
os.chdir('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/sorter_workspace/block_rec0005')
waveform_good = waveforms1min.select_units(non_redundant_templates,new_folder='waveforms_good_rec0005')
waveform_good.run_extract_waveforms()

In [ ]:
print(waveforms_good.unit_ids)

In [ ]:
template1 = waveforms_good.get_template(unit_id = 8)
template1 = template1.transpose()
print(template1.shape)

In [ ]:
locations = waveforms_good.get_channel_locations()



In [ ]:
waveforms_good.run_extract_waveforms()

In [ ]:
%matplotlib widget
import spikeinterface.widgets as sw
import spikeinterface.postprocessing as sp
import matplotlib.pyplot as plt
locations = sp.compute_unit_locations(waveforms_good)
print(locations.shape)
#ax = plt.subplot(111)
sw.plot_unit_waveforms(waveforms_good, unit_ids=[8])
print(locations)

In [ ]:
metrics = msp.get_quality_metrics(waveforms1min)
metrics

In [ ]:
good_units = msp.remove_violated_units(metrics)
print(good_units)

In [ ]:
recording,rec_name = msp.get_data_maxwell(file_path,6)
channel_location_dict = msp.get_channel_locations_mapping(recording)
print(channel_location_dict)

In [ ]:
import spikeinterface.full
unit_extremum_channel =spikeinterface.full.get_template_extremum_channel(waveforms1min, peak_sign='neg')
    #Step 1: keep only units that are in good_units 
unit_extremum_channel = {key:value for key,value in unit_extremum_channel.items() if key in good_units}

In [ ]:
# New dictionary with combined information
new_dict = {}
for template, channel in unit_extremum_channel.items():
    if template not in new_dict:
        new_dict[template] = {}

# Add an entry for this template and its corresponding location to the new dictionary
    new_dict[template][channel] = [int(channel_location_dict[channel][0]/17.5),int(channel_location_dict[channel][1]/17.5)]
print(new_dict)

In [ ]:
import axon_velocity as av

gtr = av.compute_graph_propagation_velocity(template=template1, locations=locations, fs=20000)

In [ ]:
templates = helper.get_templates_with_same_channels('/home/mmpatil/Documents/spikesorting/MEA_Analysis/Python/Electrodes_rec0007')
print(templates)

In [ ]:
job_kwargs = dict(n_jobs=64, chunk_duration="1s", progress_bar=True)
waveforms_good.run_extract_waveforms(**job_kwargs)
msp.analyse_waveforms_sigui(waveforms_good)

In [ ]:
%matplotlib widget

In [ ]:
import spikeinterface.postprocessing as sp

locations = sp.compute_unit_locations(waveforms1min)
print(locations)
import numpy as np
#np.savetxt("unitloc_10mins.txt",locations)
ax = plt.subplot(111)
sw.plot_probe_map(recording,ax=ax,with_channel_ids=False)
for x,y in locations:
    ax.scatter(x,y)


In [ ]:
channel_locations = recording_chunk.get_channel_locations()
channel_ids = recording_chunk.get_channel_ids()
channel_locations_mappings= {channel_id: location for location, channel_id in zip(channel_locations, channel_ids)}
print(channel_locations_mappings)



In [ ]:
#thinking the unique locations have all the required electrodes.

required_electrodes= {channels :[int(locations[0]/17.5),int(locations[1]/17.5)] for channels, locations in channel_locations_mappings.items() if channels in unique_channels }
print(required_electrodes)

In [ ]:
import json

with open('electrode_locations_4min.json','w') as fileptr:
    json.dump(required_electrodes,fileptr)



In [ ]:
with open('electrode_locations_4min.json',) as fileptr:
    data = json.load(fileptr)
print(data)

In [ ]:
selected_electrodes = [220 * locations[1] + locations[0] for locations in data.values()]
print(selected_electrodes)

In [ ]:
val = recording_chunk.get_electrode_info
print(val)

In [ ]:
def electrode_rectangle_indices(xmin, ymin, xmax, ymax):
    return [220 * y + x for y in range(max(ymin, 0), min(ymax + 1, 120)) for x in range(max(xmin, 0), min(xmax + 1, 220))]


def electrode_rectangle_um(xmin, ymin, xmax, ymax):
    return electrode_rectangle_indices(int(xmin / 17.5), int(ymin / 17.5), int(xmax / 17.5), int(ymax / 17.5))

XMULTIPLIER = 44
YMULTIPLIER = 20

for y in range(6):
    for x in range(5):
        #print("(",XMULTIPLIER*x,YMULTIPLIER*y,XMULTIPLIER*x+XMULTIPLIER-1,YMULTIPLIER*y+YMULTIPLIER-1,")")
        print(electrode_rectangle_indices(XMULTIPLIER*x,YMULTIPLIER*y,XMULTIPLIER*x+XMULTIPLIER-1,YMULTIPLIER*y+YMULTIPLIER-1))

# print(electrode_rectangle_indices(218,0,219,1))

In [ ]:
import datetime as datetime

now = datetime.datetime.now()
strfmt = now.strftime("%Y%m%d_%H_%M_%S")
print(strfmt)

In [ ]:
import maxlab.saving

obj = maxlab.saving.Saving()

obj.start()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))
spike_times = {}
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=1*fs,end_frame=100*fs)
    print(spike_train)
    if len(spike_train) > 0:
        spike_times[idx] = spike_train / float(fs)
        #print(spike_times[unit_id])
       # print(unit_id*np.ones_like(spike_times[unit_id]))
        ax1.plot(spike_times[idx],idx*np.ones_like(spike_times[idx]),
                             marker='|', mew=1, markersize=3,
                             ls='',color='black')
                       

In [ ]:
t_start = 0 
t_end = int(600*fs)
dt = 1
#initialising the spike train.
units= clean_sorting.get_num_units()
frame_numbers = t_end
spike_array = np.zeros((units,frame_numbers), dtype= int)
for idx, unit_id in enumerate(clean_sorting.get_unit_ids()):
    spike_train = clean_sorting.get_unit_spike_train(unit_id,start_frame=t_start,end_frame=t_end)
    for spike_time in spike_train:
        spike_array[idx,spike_time] = 1

print(spike_array)

print(spike_array[0,63782])

In [ ]:
np.savez_compressed('spike_array_compressed_blockactivity.npz',spike_array)

In [ ]:
with np.load('spike_array_compressed_blockactivity.npz') as data:
    decompressed_data = data['arr_0']

print(np.array_equal(spike_array, decompressed_data))


In [ ]:
extremum_channels_ids =spikeinterface.full.get_template_extremum_channel(waveforms, peak_sign='neg')
print(extremum_channels_ids)



In [ ]:
colors = [ 'Lime','Gold', 'Orange','Orangered']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([34, 49]):
    wf = waveforms.get_waveforms(unit_id)
    color = colors[i]
    ax.plot(wf[:, :,channel_association_dict[594]].T, color=color, lw=0.3)
print(wf.shape)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
wf=[]
for i, unit_id in enumerate([218]):
    wf = waveform_good.get_waveforms(unit_id)
    ax.plot(wf[:, :,channel_association_dict[902]].T, color=colors[0], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[613]].T, color=colors[1], lw=0.3)
    ax.plot(wf[:, :,channel_association_dict[663]].T, color=colors[2], lw=0.3)
print(wf.shape)

In [ ]:
peak_shift=si.get_template_extremum_channel_peak_shift(waveform_good)

print(peak_shift)

In [ ]:
colors = [ 'Fuchsia','Olive', 'Teal']
fig, ax = plt.subplots()
for i, unit_id in enumerate([26, 40 , 46]):
    template = waveforms.get_template(unit_id)
    color = colors[i]
    ax.plot(template[:, channel_association_dict[780]].T, color=color, lw=3)
print(template.shape)

In [ ]:
w = sw.plot_unit_templates(waveform_good, unit_ids=[183],plot_channels=False )

In [ ]:
w = sw.plot_unit_waveforms(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_templates(waveform_good, unit_ids=[2,4,7])
w = sw.plot_unit_probe_map(waveform_good, unit_ids=[2,4,7])

In [ ]:
    all_units = sortingKS3.get_unit_ids()
    print(f"Total units counts {len(all_units)}")
    print(f"Units : {all_units}")
    rp_contamination,rp_violation = qm.compute_refrac_period_violations(waveforms)
    print(f"rp_contaminaiton : {rp_contamination}")

    refrct_violated_units = [unit for unit,ratio in rp_contamination.items() if ratio >0.0]

    snr_ratio = qm.compute_snrs(waveforms)
    print(f"SNR ratio : {snr_ratio}")
    snr_violated_units = [unit for unit,ratio in snr_ratio.items() if ratio >5]

    firing_rate = qm.compute_firing_rates(waveforms)
    print(f"Firing rate : {firing_rate}")
    fr_violated_units = [unit for unit,ratio in firing_rate.items() if ratio <=0.1]

    violated_units = list(set(refrct_violated_units+fr_violated_units))
    print(f"violated units {violated_units} removed")

    
    good_units = [units for units in all_units if units not in violated_units ]